In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import kraft

In [3]:
na_pa = {}

na_se_li_ = {}

for pa in kraft.path.get_child("../raw/"):

    if pa[-3:] != ".md":

        continue

    na = pa.split("/")[-1][: -len(".md")]

    assert na == kraft.string.untitle(na)

    na_pa[na] = pa

    # print("-" * 80)
    
    # print(na)

    se_li_ = {
        "~": [],
        ".": [],
        "<": [],
        "md<": [],
        ">": [],
        "md>": [],
    }
    
    se_ = []

    with open(pa) as io:

        for li in io:

            li = li.strip()

            if li == "":

                continue

            if li[0] == "#":

                se = li[2]
                
                se_.append(se)

                continue

            if li[0] == "(":

                assert li[-2] == "."

                continue

            se_li_[se].append(li)
    
    assert all(se in se_ for se in [".", "<", ">"])

    assert any(0 < len(li_) for li_ in se_li_.values())

    na_se_li_[na] = se_li_

len(na_se_li_)

243

In [4]:
for na, se_li_ in na_se_li_.items():

    for li in se_li_["<"]:

        assert kraft.string.untitle(li) not in na_se_li_, "{}: {}".format(na, li)

In [5]:
import os
import shutil

In [6]:
for na, se_li_ in na_se_li_.items():

    for li in se_li_[">"]:

        limi = kraft.string.untitle(li)

        if limi in na_se_li_:

            na_se_li_[limi]["md<"].append(na)

            se_li_[">"].remove(li)

            se_li_["md>"].append(limi)

In [7]:
import pandas as pd

In [8]:
di = "../compiled/"

if os.path.isdir(di):

    shutil.rmtree(di)

de = "\n\n"

for na, se_li_ in na_se_li_.items():
    
    #print("-" * 80)
    
    #print(na)

    pa = na_pa[na].replace("../raw/", di)

    kraft.path.make(pa, pr=False)

    with open(pa, "w") as io:
        
        ca_ = [li.upper() for li in se_li_["md<"]]

        fu_ = []

        fu_ += se_li_[">"]

        for li in se_li_["md>"]:
            
            fu_.append(li.upper())

            fu_ += na_se_li_[li][">"]

        for se, li_ in [
            ["Comment", se_li_["."]],
            ["Epidemiology", se_li_["<"]],
            ["Cause", ca_],
            ["Symptom and Sign", fu_],
        ]:

            io.write("# {}".format(se) + de)

            if len(li_) == 0:

                continue

            for li, n in pd.value_counts(li_).items():
                
                if 1 < n:
                    
                    bbb = [na]
                    
                    bbb += [limd for limd in se_li_["md>"] if li in na_se_li_[limd][">"]]
                    
                    print("{}:\n\t{}\n".format(li, "\n\t".join(bbb)))
                    
            io.write(de.join(li_) + de)

Hepatocellular carcinoma:
	viral_hepatitis
	chronic_hepatitis

Pain in abdomen:
	diabetes_mellitus
	diabetic_ketoacidosis
	gastroesophageal_reflux_disease

Fatigue:
	diabetes_mellitus
	chronic_kidney_disease

Nausea:
	diabetes_mellitus
	diabetic_ketoacidosis
	chronic_kidney_disease

Mental status change:
	diabetic_ketoacidosis
	cerebral_edema

Short stature:
	hypopituitarism
	hypothyroidism
	growth_hormone_deficiency

Fatigue:
	hypopituitarism
	hypothyroidism
	growth_hormone_deficiency

Nodular goiter:
	follucular_thyroid_carcinoma
	anaplastic_thyroid_carcinoma

Nodular goiter:
	papillary_thyroid_carcinoma
	anaplastic_thyroid_carcinoma

Infertility:
	hypothyroidism
	hyperprolactinemia

Delayed puberty:
	hypothyroidism
	hyperprolactinemia

High blood insulin:
	neuroendocrine_tumor
	carcinoid
	insulinoma

Flu-like:
	adenovirus
	viral_hepatitis

Flu-like:
	hepatitis_a_virus
	viral_hepatitis

Flu-like:
	hepatitis_b_virus
	viral_hepatitis

Flu-like:
	hepatitis_c_virus
	viral_hepatitis

Flu-

In [9]:
le_ = []

ri_ = []

for na, se_li_ in na_se_li_.items():

    le_ += se_li_["<"]

    ri_ += se_li_[">"]

for li_ in [le_, ri_]:

    print("-" * 80)

    li_n = pd.value_counts(li_)

    print(li_n)

--------------------------------------------------------------------------------
Common                  21
Adult                   18
Young adult             11
Elderly                 10
Obesity                  9
                        ..
Genome AR                1
Use of psychotropic      1
Genome GHR               1
High sexual activity     1
Use of isoniazid         1
Length: 157, dtype: int64
--------------------------------------------------------------------------------
Fatigue                               21
Flu-like                              10
Nausea                                 9
Short stature                          9
Infertility                            9
                                      ..
Blood insulin-like growth factor 1     1
Blood follicle stimulating hormone     1
Osmotic demyelination syndrome         1
Retinopathy                            1
High blood albumin                     1
Length: 454, dtype: int64
